In [56]:
import matplotlib.pyplot as plt
from workflow.scripts.plotting_tools import create_facet_plot
import xarray as xr
from workflow.scripts.utils import regrid_global
from workflow.scripts.plotting_tools import global_map
import matplotlib as mpl

In [19]:
nmodels = len(snakemake.input.path_ctrl)
ctrl_path = snakemake.input.path_ctrl
ctrl_path=sorted(ctrl_path)
exp_path = snakemake.input.path_exp
exp_path=sorted(exp_path)

In [112]:
diffs = {}
for ctrl, exp in zip(ctrl_path, exp_path):
    ds_exp = xr.open_dataset(exp).load().mean(dim='year', keep_attrs=True)
    ds_ctrl = xr.open_dataset(ctrl).load().mean(dim='year',keep_attrs=True)
    diff_ds = ds_exp.copy()
    with xr.set_options(keep_attrs=True):
        diff = (ds_exp[ds_exp.variable_id].sum(dim='lev') -
                ds_ctrl[ds_exp.variable_id].sum(dim='lev'))/(ds_ctrl[ds_exp.variable_id].sum(dim='lev'))
        diff['units'] = '%'
        diff_ds = diff_ds.assign({diff_ds.varialbe_id : diff*100})
        
    #diff_ds = diff_ds.sum(dim='lev', keep_attrs=True)
    diffs[ds_exp.source_id] = diff_ds
    

In [113]:
fig,ax = create_facet_plot(nmodels, figsize=(14,12))
cmap = mpl.cm.bwr
norm = mpl.colors.Normalize(vmin=-40,vmax=40)
for ds, axi in zip(diffs, ax):
    temp_ds = diffs[ds]
    temp_ds[temp_ds.variable_id].plot(ax=ax[axi], cmap=cmap, norm=norm,  add_colorbar=False)
    ax[axi].set_title(ds)
    global_map(ax[axi])

cax = fig.add_axes([0.94,0.2,0.02,0.62])
fig.colorbar(mpl.cm.ScalarMappable(norm, cmap=cmap), cax=cax, extend='max',
             label='$\Delta$[CDNC] %')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', dpi=144, facecolor='w')
